In [3]:
import numpy as np
from scipy.integrate import dblquad
from scipy.optimize import curve_fit
from scipy.optimize import fsolve
from tqdm import tqdm
import os
import re
import matplotlib.pyplot as plt
plt.rcParams.update({
    "text.usetex": True,
    "font.family": "Computer Modern",
    "figure.dpi": 100,
})
%matplotlib widget

class fixed_values:
  def __init__(self):
    self.h=4.135667e-9 #eV us
    self.hbar=self.h/(2*np.pi) #eV us rad-1
    self.c=299792458 #um/us
    self.k_B=8.617343e-5 #eV/K
    self.eta_pb_max=0.59
    self.energy_au=27.211386 #eV
    self.time_au=2.418884e-11 #us
    self.length_au=5.291772e-5 #um
consts = fixed_values()

In [4]:
length = 90
width = 10
height = 0.04

v_L = 4.09e3 #um/us

def distfunc(phi,theta):
    return np.min(np.stack([height/2/np.sqrt(1-np.sin(phi)**2),length/2/np.sqrt(1-np.sin(theta)**2),width/2/np.sqrt(1-np.sin(np.pi/2-theta)**2)],axis=-1),axis=-1)

mfp = dblquad(distfunc,0,np.pi/2,0,np.pi/2)[0]*4/np.pi**2

D_ph = mfp*v_L/3

print(D_ph, '= D (um2/us)')
mfp

136.0405665478132 = D (um2/us)


0.09978525663653781

In [5]:
#Al phonon times
nue = 17.2e9 #eV-1 um-3
D=15000 #um2 us-1
Tc = 1.11 #K
Delta = 1.768*consts.k_B*Tc #eV
omega_D = 9.66e7 #in MHz rad
T_D = consts.hbar*omega_D/consts.k_B #K

# energy per particle, consts.k_B*T_D (Debye), 3*Delta (E2)
E=consts.k_B*T_D

omega = E/consts.hbar #rad/us = MHz rad
f = omega/(2*np.pi) # MHz
T = 0.2 #K

def calcN():
    rho_m = 17.1 #g cm-3
    Z = 2 #valence electrons per atom
    m_a = 180.94788 #u
    N_A = 6.022e23 #u/g
    return N_A * Z * rho_m / m_a * 1e-12 # electrons um-3
N=calcN()

rho_m = 17.1*5.61e32/consts.c**2 *1e-12#eV c-2 um-3

v_L = 4.09e3 #um/us

p_fermi = np.sqrt((consts.hbar)**2*(3*np.pi**2*N)**(2/3))
k_fermi = p_fermi/consts.hbar

mstar = 2*consts.hbar**2*nue*np.pi**2/k_fermi # from suppl

tau_elastic = nue*D*mstar/N # <= Einstein relation (sigma = e^2 N(0) D) and drude effective mass, tau elastic time
v_fermi = p_fermi/mstar

l=v_fermi*tau_elastic
k=omega/v_L
print(k*l, '= kl >> 1 ?')

alpha_L = N*mstar/(rho_m*v_L*tau_elastic) * (1/3 * (k**2*l**2*np.arctan(k*l)/(k*l-np.arctan(k*l))) - 1)
print(alpha_L, '= alpha (/um)')

# alpha_corr = alpha_L * 2/(np.exp(Delta/(consts.k_B*T))+1)
# print(alpha_corr) # nvt omdat op hoge energieën / lage temperaturen attenuation kan door cooper paren te breken, wat ongv gelijk is aan de normal electron density.

tau_ph = 1/(v_L*alpha_L)
print(tau_ph, '= tau_ph-e (us)')

D_ph = v_L**2*tau_ph/3
print(D_ph, '= D (um2/us)')

print(v_L*tau_ph)

528.3284283806961 = kl >> 1 ?
31.758991174537467 = alpha (/um)
7.698568766319552e-06 = tau_ph-e (us)
42.927476059956696 = D (um2/us)
0.03148714625424697


In [6]:
#bTa phonon times
nue = 30.3e9 #eV-1 um-3
D=50 #um2 us-1
T_D = 221 #K
Tc = 0.87 #K
Delta = 1.768*consts.k_B*Tc #eV
omega_D = consts.k_B*T_D/consts.hbar #in MHz rad

# energy per particle, consts.k_B*T_D (Debye), 3*Delta (E2)
E=2*Delta

omega = E/consts.hbar #rad/us = MHz rad
f = omega/(2*np.pi) # MHz
T = 0.2 #K

def calcN():
    rho_m = 17.1 #g cm-3
    Z = 2 #valence electrons per atom
    m_a = 180.94788 #u
    N_A = 6.022e23 #u/g
    return N_A * Z * rho_m / m_a * 1e-12 # electrons um-3
N=calcN()

rho_m = 17.1*5.61e32/consts.c**2 *1e-12#eV c-2 um-3

v_L = 4.09e3 #um/us

p_fermi = np.sqrt((consts.hbar)**2*(3*np.pi**2*N)**(2/3))
k_fermi = p_fermi/consts.hbar

mstar = 2*consts.hbar**2*nue*np.pi**2/k_fermi # from suppl

tau_elastic = nue*D*mstar/N # <= Einstein relation (sigma = e^2 N(0) D) and drude effective mass, tau elastic time
v_fermi = p_fermi/mstar

l=v_fermi*tau_elastic
k=omega/v_L
print(k*l, '= kl >> 1 ?')

alpha_L = N*mstar/(rho_m*v_L*tau_elastic) * (1/3 * (k**2*l**2*np.arctan(k*l)/(k*l-np.arctan(k*l))) - 1)
print(alpha_L, '= alpha (/um)')

# alpha_corr = alpha_L * 2/(np.exp(Delta/(consts.k_B*T))+1)
# print(alpha_corr) # nvt omdat op hoge energieën / lage temperaturen attenuation kan door cooper paren te breken, wat ongv gelijk is aan de normal electron density.

tau_ph = 1/(v_L*alpha_L)
print(tau_ph, '= tau_ph-e (us)')

D_ph = v_L**2*tau_ph/3
print(D_ph, '= D (um2/us)')

tau_ph*v_L

# check tau_ee vs tau_F
epsilonF = (consts.hbar)**2/(2*mstar)*(3*np.pi**2*N)**(2/3)
rs = 1.2947e-4
tau_e_e = 1/((0.58**2 / (consts.hbar*epsilonF))*rs**0.5 / 7.96)
print(tau_elastic, '=tau_elastic vs tau_e-e=', tau_e_e)

0.012934777868070773 = kl >> 1 ?
0.0008738587064492905 = alpha (/um)
0.2797921170798572 = tau_ph-e (us)
1560130.1712411863 = D (um2/us)
2.300514173734026e-10 =tau_elastic vs tau_e-e= 3.856294505883355e-06


In [7]:
#Al, put everything in eV, us and K
nue = 17.2e9 #eV-1 um-3
OmegaD = 36.9e-3 #eV
tau0 = 0.11 #us
Tc = 1.19 #K
c = 6420 #um/us
#Delta = 1.768*consts.k_B*Tc #eV
Delta = 0.17e-3#eV
E1 = 67*OmegaD

lambda0= (1/(OmegaD*tau0))*(OmegaD/(consts.k_B*Tc))**3 #eV-1 us-1
tau1 = (3/(lambda0*OmegaD))*(E1/OmegaD)
print('tau1 = ',tau1*1e6,'ps')


nuph = 3*OmegaD**2/(2*np.pi**2*c**3)/consts.hbar**3
lambda1 = (lambda0*nue/(2*nuph)) #eV-1 us-1
Omega1 = OmegaD*np.sqrt(3*lambda1/lambda0) #eV
t=1/(lambda1*Omega1) #us
tI = 2*t
print('tI = ',tI*1e6,'ps')

omega = 4.5*Delta
taus = tau0/((Delta/(consts.k_B*Tc))**3*(1/3*((omega/Delta)**2-1)**(3/2)+5/2*((omega/Delta)**2-1)**(1/2)-Delta/(2*omega)*(1+4*(omega/Delta)**2)*np.log(omega/Delta+((omega/Delta)**2-1)**(1/2))))
print('tII = ', taus*1e3,'ns')

tau1 =  0.4745376414608857 ps
tI =  15.524206835736628 ps
tII =  1.2563689598573353 ns


In [12]:
#bTa, put everything in eV, us and K
nue = 30.3e9 #eV-1 um-3
TD = 221 # debye temp in K
OmegaD = consts.k_B*TD
tau0=0.150 #us
Tc = 0.87 #K
c = 1730 #um/us
Delta = 1.768*consts.k_B*Tc #eV
rs = 1.2947e-4 #um # based on n_e => spherical volume => radius
rs_au = rs/consts.length_au
n = 1.1e11 # um-3
D=50 #um2 us-1

omega = 4.5*Delta
taus = tau0/((Delta/(consts.k_B*Tc))**3*(1/3*((omega/Delta)**2-1)**(3/2)+5/2*((omega/Delta)**2-1)**(1/2)-Delta/(2*omega)*(1+4*(omega/Delta)**2)*np.log(omega/Delta+((omega/Delta)**2-1)**(1/2))))
tII = taus

mstar = 2*consts.hbar**2*nue*np.pi**2/k_fermi # from suppl
epsilonF = (consts.hbar)**2/(2*mstar)*(3*np.pi**2*n)**(2/3)
print(epsilonF)

lambda0= (1/(OmegaD*tau0))*(OmegaD/(consts.k_B*Tc))**3
lambda_au = (1/(OmegaD*tau0))*(OmegaD/(consts.k_B*Tc))**3 * consts.energy_au*consts.time_au
E1 = 2.82*OmegaD*rs_au**(-1/4)*(lambda_au/3 * epsilonF/OmegaD)**(1/2)

tau1 = (3/(lambda0*OmegaD))*(E1/OmegaD)

nuph = 3*OmegaD**2/(2*np.pi**2*c**3)/(consts.h/(2*np.pi))**3
lambda1 = (lambda0*nue/(2*nuph)) #eV-1 us-1
Omega1 = OmegaD*np.sqrt(3*lambda1/lambda0) #eV
print(Omega1)
Omega1 = 8.83e-4
Omega1 = 4.25e-4
Omega1 = 3.54e-4
t=1/(lambda1*Omega1) #us
tI = 2*t

print('tau1 = ',tau1*1e6,'ps')
print('tI = ',tI*1e6,'ps')
print('tII = ', tII*1e3,'ns')

2.7539214220494648
0.0006644906629659631
tau1 =  0.8352121409468417 ps
tI =  2426.2713984132456 ps
tII =  1.4123903146888779 ns


In [13]:
(8.83e-4 + 4.25e-4)/2

0.000654